In [1]:
import urllib.request
from pathlib import Path

BASE_URL = "https://raw.githubusercontent.com/FelixZhan/AtyAN/main/"
HELPER_FILES = [
    "requirements.txt",
    "BP1234-ONSET-WCOND-NUMID.csv",
    "BP1234-ONSET.csv",
]

for filename in HELPER_FILES:
    dest = Path(filename)
    if dest.exists():
        print(f"{filename} already present, skipping download.")
        continue
    print(f"Downloading {filename}...")
    urllib.request.urlretrieve(f"{BASE_URL}{filename}", dest)

print("Helper files are ready.")


requirements.txt already present, skipping download.
BP1234-ONSET-WCOND-NUMID.csv already present, skipping download.
Helper files are ready.


In [2]:
!pip install -q -r requirements.txt
!pip install -q imbalanced-learn


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import (
    StratifiedKFold,
    GridSearchCV,
    cross_validate,
    GroupKFold,
    GroupShuffleSplit,
    LeaveOneGroupOut,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    precision_score,
    recall_score,
    balanced_accuracy_score,
    confusion_matrix,
    make_scorer,
    average_precision_score,
    precision_recall_curve,
)
from sklearn.base import clone

import statsmodels.formula.api as smf
import statsmodels.api as sm

from imblearn.ensemble import BalancedRandomForestClassifier

# Inline replacements for analysis_utils (prefer NUMID dataset)
DATA_FILE_PREFERRED = Path("BP1234-ONSET-WCOND-NUMID.csv")
DATA_FILE_FALLBACK = Path("BP1234-ONSET.csv")
PERSISTENCE_WAVES = [1, 4, 5, 6]
RISK_SUFFIXES = ["tii", "bs", "dres", "socf", "dep", "intbmi"]
PRODROMAL_FEATURE_NAMES = ["BE", "CB", "WSO", "FEAR", "FAT", "LEB"]
CB_COLUMN_SUFFIXES = ["ed8a", "ed9a", "ed10a", "ed11a"]

def load_base_dataset(columns=None):
    for candidate in (DATA_FILE_PREFERRED, DATA_FILE_FALLBACK):
        if candidate.exists():
            print(f"Using dataset: {candidate.name}")
            return pd.read_csv(candidate, usecols=columns, low_memory=False)
    raise FileNotFoundError("BP1234-ONSET-WCOND-NUMID.csv or BP1234-ONSET.csv is required.")

def _coerce_numeric(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series, errors="coerce")

def _coerce_wave_risk_features(work: pd.DataFrame, wave: int) -> list:
    cols = []
    for suffix in RISK_SUFFIXES:
        col = f"w{wave}{suffix}"
        if col not in work.columns:
            continue
        work[col] = _coerce_numeric(work[col])
        cols.append(col)
    return cols

def _engineer_wave_prodromals(work: pd.DataFrame, wave: int) -> list:
    cols = []
    base = pd.Series(np.nan, index=work.index)
    be_src = f"w{wave}ede1a"
    work[f"BE_w{wave}"] = _coerce_numeric(work.get(be_src, base))
    cols.append(f"BE_w{wave}")

    cb_candidates = [f"w{wave}{suffix}" for suffix in CB_COLUMN_SUFFIXES]
    cb_present = [c for c in cb_candidates if c in work.columns]
    if cb_present:
        cb_block = work[cb_present].apply(_coerce_numeric)
        work[f"CB_w{wave}"] = cb_block.max(axis=1, skipna=True)
        work.loc[cb_block.notna().sum(axis=1) == 0, f"CB_w{wave}"] = np.nan
    else:
        work[f"CB_w{wave}"] = np.nan
    cols.append(f"CB_w{wave}")

    work[f"WSO_w{wave}"] = _coerce_numeric(work.get(f"w{wave}ed15a", base))
    work[f"FEAR_w{wave}"] = _coerce_numeric(work.get(f"w{wave}ed17a", base))
    work[f"FAT_w{wave}"] = _coerce_numeric(work.get(f"w{wave}ed19a", base))
    cols.extend([f"WSO_w{wave}", f"FEAR_w{wave}", f"FAT_w{wave}"])

    mbmi_pct = _coerce_numeric(work.get(f"w{wave}mbmi_pct", base))
    work[f"LEB_w{wave}"] = np.clip(90.0 - mbmi_pct, 0, None) / 90.0
    cols.append(f"LEB_w{wave}")
    return cols

def engineer_baseline_features(df: pd.DataFrame):
    work = df.copy()
    risk_cols_by_wave = {}
    prodromal_cols_by_wave = {}
    for wave in PERSISTENCE_WAVES:
        risk_cols_by_wave[wave] = _coerce_wave_risk_features(work, wave)
        prodromal_cols_by_wave[wave] = _engineer_wave_prodromals(work, wave)

    anova_features = list(
        dict.fromkeys(risk_cols_by_wave.get(1, []) + prodromal_cols_by_wave.get(1, []))
    )
    feature_sets = {
        "risk": risk_cols_by_wave.get(1, []),
        "prodromal": prodromal_cols_by_wave.get(1, []),
        "anova_features": anova_features,
        "onset_features": [],
        "persistence_features": [],
        "all_features": anova_features,
        "outcomes": anova_features,
    }
    return work, feature_sets


**Aim 3: Persistence vs remission of AAN over sequential 1-year intervals**
- Baseline = first AAN onset wave (1-5). Wave 6 first-onset cases are excluded (no follow-up to assess persistence).
- Follow-up sequence by baseline: wave 1 -> check wave 4, then 5, then 6; wave 2 or 3 -> check waves 4, then 5, then 6; wave 4 -> check waves 5, then 6; wave 5 -> check wave 6.
- We evaluate follow-ups in order and keep only one interval per person: the earliest interval where both ed14 and ed16 are present; if any such interval shows ed14 & ed16 >= 5, classify as persistent (1) and stop. If none are persistent but at least one interval was evaluable, classify remission (0) using the latest evaluable follow-up. If no follow-up has both ed14 and ed16, discard the case.
- Wave mapping: wave 4 ~ year 1, wave 5 ~ year 2, wave 6 ~ year 3 for the purposes of these 1-year intervals.
- Predictors: baseline_tii, baseline_bs, baseline_dres, baseline_socf, baseline_dep (risk factors) plus baseline_BE, baseline_CB, baseline_WSO, baseline_FEAR, baseline_FAT, baseline_LEB (prodromal symptoms), standardized across waves.


In [4]:
raw_df = load_base_dataset()
feature_df, feature_sets = engineer_baseline_features(raw_df)

print(f"Raw dataset shape: {raw_df.shape}")
print(f"Feature matrix shape: {feature_df.shape}")


Using dataset: BP1234-ONSET-WCOND-NUMID.csv
Raw dataset shape: (1952, 3715)
Feature matrix shape: (1952, 3739)


In [5]:
# Condition cleaning and dummy coding (BP vs Control vs Healthy Weight)
COND_MAP = {
    "peer delivered": "BP",
    "ebody": "BP",
    "clincian delivered": "BP",
    "clinician delivered": "BP",
    "diss. (bp)": "BP",
    "exp writing": "BP",
    "control/video control": "Control",
    "healthy weight": "Healthy Weight",
}

COND_CANONICAL = {
    "BP": "BP",
    "Control": "Control",
    "Healthy Weight": "Healthy Weight",
}

def clean_and_encode_condition(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean 'study_cond' into three levels and add two dummy vectors:
      - cond_bp: 1 = BP trial condition, 0 = Control/Healthy Weight
      - cond_hw: 1 = Healthy Weight, 0 = Control/BP
    Control is the reference (0,0).
    Raises if there are missing/unmapped values.
    """
    if "study_cond" not in df.columns:
        raise KeyError("Missing 'study_cond' column; use dataset with condition labels.")

    cond_raw = df["study_cond"]
    if cond_raw.isna().any() or cond_raw.astype(str).str.strip().eq("").any():
        raise ValueError("Found missing/blank entries in 'study_cond'; expected none.")

    cond_norm = cond_raw.astype(str).str.strip().str.lower()
    cond_clean = cond_norm.map(COND_MAP)

    if cond_clean.isna().any():
        bad_vals = sorted(cond_raw.loc[cond_clean.isna()].unique())
        raise ValueError(f"Unmapped 'study_cond' values: {bad_vals}")

    out = df.copy()
    out["cond_clean"] = cond_clean.map(COND_CANONICAL).astype("category")
    out["cond_bp"] = (out["cond_clean"] == "BP").astype(int)
    out["cond_hw"] = (out["cond_clean"] == "Healthy Weight").astype(int)
    return out

raw_df = clean_and_encode_condition(raw_df)
feature_df = clean_and_encode_condition(feature_df)


In [6]:
# Unique participant ID column - change if needed
ID_COL = "id"  # e.g., "study_id" if that's what your data uses

# AAN presence columns used for Aim 3 baselines (include all waves 1-6)
# Wave mapping: wave 5 = year 2, wave 6 = year 3
AAN_PRESENCE_COLS = {
    1: "w1ONSET-FULL",
    2: "w2ONSET-FULL-mBMI",
    3: "w3ONSET-FULL-mBMI",
    4: "w4ONSET-FULL-mBMI",
    5: "w5ONSET-FULL-mBMI",
    6: "w6ONSET-FULL-mBMI",
}

print("Using AAN presence columns (baselines):", AAN_PRESENCE_COLS)
print("ID column:", ID_COL)


Using AAN presence columns (baselines): {1: 'w1ONSET-FULL', 2: 'w2ONSET-FULL-mBMI', 3: 'w3ONSET-FULL-mBMI', 4: 'w4ONSET-FULL-mBMI', 5: 'w5ONSET-FULL-mBMI', 6: 'w6ONSET-FULL-mBMI'}
ID column: id


In [7]:
# Aim 3 configuration
FOLLOWUP_MAP = {
    1: 4,  # wave 1 onset -> check ed14/ed16 at wave 4,
    4: 5,  # wave 4 onset -> check ed14/ed16 at wave 5
    5: 6,  # wave 5 onset -> check ed14/ed16 at wave 6
    # waves 2, 3, 4, 6: no later follow-up; treated as remission
}

# Standardized predictor names shared across intervals
RISK_PREDICTORS = [
    "baseline_tii",
    "baseline_bs",
    "baseline_dres",
    "baseline_socf",
    "baseline_dep",
    "baseline_bmi"
]
PRODROMAL_PREDICTORS = [
    "baseline_BE",
    "baseline_CB",
    "baseline_WSO",
    "baseline_FEAR",
    "baseline_FAT",
    "baseline_LEB"
]
AIM3_PREDICTORS = RISK_PREDICTORS + PRODROMAL_PREDICTORS

print("Follow-up map (baseline -> follow-up):", FOLLOWUP_MAP)
print("Predictors (risk + prodromal):", AIM3_PREDICTORS)


Follow-up map (baseline -> follow-up): {1: 4, 4: 5, 5: 6}
Predictors (risk + prodromal): ['baseline_tii', 'baseline_bs', 'baseline_dres', 'baseline_socf', 'baseline_dep', 'baseline_bmi', 'baseline_BE', 'baseline_CB', 'baseline_WSO', 'baseline_FEAR', 'baseline_FAT', 'baseline_LEB']


In [8]:
def _numeric(series: pd.Series, index: pd.Index) -> pd.Series:
    if series is None:
        return pd.Series(np.nan, index=index)
    return pd.to_numeric(series, errors="coerce")

def extract_baseline_predictors(df: pd.DataFrame, wave: int) -> pd.DataFrame:
    base_index = df.index
    predictors = pd.DataFrame(index=base_index)

    # Risks
    predictors["baseline_tii"] = _numeric(df.get(f"w{wave}tii"), base_index)
    predictors["baseline_bs"] = _numeric(df.get(f"w{wave}bs"), base_index)
    predictors["baseline_dres"] = _numeric(df.get(f"w{wave}dres"), base_index)
    predictors["baseline_socf"] = _numeric(df.get(f"w{wave}socf"), base_index)
    predictors["baseline_dep"] = _numeric(df.get(f"w{wave}dep"), base_index)
    predictors["baseline_bmi"] = _numeric(df.get(f"w{wave}intbmi"), base_index)
    # Prodromals
    predictors["baseline_BE"] = _numeric(df.get(f"w{wave}ede1a"), base_index)

    cb_candidates = [f"w{wave}ed{num}a" for num in (8, 9, 10, 11)]
    cb_present = [c for c in cb_candidates if c in df.columns]
    if cb_present:
        cb_block = df[cb_present].apply(pd.to_numeric, errors="coerce")
        predictors["baseline_CB"] = cb_block.max(axis=1, skipna=True)
        predictors.loc[cb_block.notna().sum(axis=1) == 0, "baseline_CB"] = np.nan
    else:
        predictors["baseline_CB"] = np.nan

    predictors["baseline_WSO"] = _numeric(df.get(f"w{wave}ed15a"), base_index)
    predictors["baseline_FEAR"] = _numeric(df.get(f"w{wave}ed17a"), base_index)
    predictors["baseline_FAT"] = _numeric(df.get(f"w{wave}ed19a"), base_index)

    mbmi_pct = _numeric(df.get(f"w{wave}mbmi_pct"), base_index)
    predictors["baseline_LEB"] = np.clip(90.0 - mbmi_pct, 0, None) / 90.0

    for cond_col in ["cond_bp", "cond_hw"]:
        if cond_col in df.columns:
            predictors[cond_col] = _numeric(df.get(cond_col), base_index)

    return predictors


In [9]:
def _get_followup_scores(df: pd.DataFrame, idx: int, followup_wave: int):
    ed14_col = f"w{followup_wave}ed14"
    ed16_col = f"w{followup_wave}ed16"
    ed14 = pd.to_numeric(df.at[idx, ed14_col], errors="coerce") if ed14_col in df.columns else np.nan
    ed16 = pd.to_numeric(df.at[idx, ed16_col], errors="coerce") if ed16_col in df.columns else np.nan
    return ed14, ed16

PAIR_FOLLOWUPS = [(1, 4), (4, 5), (5, 6)]

def build_aim3_person_interval_df(df: pd.DataFrame) -> pd.DataFrame:
    predictors_by_wave = {w: extract_baseline_predictors(df, w) for w in AAN_PRESENCE_COLS}
    rows = []

    for idx, row in df.iterrows():
        for baseline_wave, followup_wave in PAIR_FOLLOWUPS:
            baseline_col = AAN_PRESENCE_COLS.get(baseline_wave)
            if baseline_col is None or baseline_col not in df.columns:
                continue

            onset_flag = pd.to_numeric(row[baseline_col], errors="coerce")
            if onset_flag != 1:
                continue

            ed14, ed16 = _get_followup_scores(df, idx, followup_wave)
            if pd.isna(ed14) or pd.isna(ed16):
                continue  # skip unevaluable pairs

            persistence = int((ed14 >= 5) and (ed16 >= 5))
            remission = 1 - persistence

            baseline_predictors = predictors_by_wave.get(baseline_wave, pd.DataFrame(index=df.index))
            if idx not in baseline_predictors.index:
                continue
            predictors = baseline_predictors.loc[[idx]]

            interval_row = {
                ID_COL: df.at[idx, ID_COL],
                "baseline_wave": baseline_wave,
                "followup_wave": followup_wave,
                "interval_label": f"{baseline_wave}_to_{followup_wave}",
                "aan_persistent_followup": persistence,
                "aan_remit_followup": remission,
            }
            for col, val in predictors.iloc[0].items():
                interval_row[col] = val
            rows.append(interval_row)

    if not rows:
        return pd.DataFrame()
    return pd.DataFrame(rows)

aim3_interval_df = build_aim3_person_interval_df(feature_df)

if aim3_interval_df.empty:
    raise ValueError("Aim 3 dataset is empty after applying baseline/follow-up rules.")

print("Aim 3 person-interval rows:", len(aim3_interval_df))
print(
    "Persistence vs remission by interval (baseline onset required; ed14 & ed16 >= 5 at follow-up):"
)
interval_summary = (
    aim3_interval_df.groupby("interval_label")["aan_persistent_followup"]
    .value_counts()
    .unstack(fill_value=0)
    .rename(columns={0: "remission", 1: "persistence"})
)
print(interval_summary)


Aim 3 person-interval rows: 70
Persistence vs remission by interval (baseline onset required; ed14 & ed16 >= 5 at follow-up):
aan_persistent_followup  remission  persistence
interval_label                                 
1_to_4                          26            7
4_to_5                          21           10
5_to_6                           5            1


In [65]:
# To run separate models for a given interval, filter aim3_interval_df before this block.
condition_covariates = [c for c in ["cond_bp", "cond_hw"] if c in aim3_interval_df.columns]
available_predictors = [c for c in AIM3_PREDICTORS if c in aim3_interval_df.columns]
available_predictors = list(dict.fromkeys(available_predictors + condition_covariates))
interval_dummies = pd.get_dummies(aim3_interval_df["interval_label"], prefix="interval", drop_first=True)

model_df = pd.concat(
    [
        aim3_interval_df[[ID_COL, "interval_label", "baseline_wave", "followup_wave", "aan_persistent_followup"]],
        aim3_interval_df[available_predictors],
        interval_dummies,
    ],
    axis=1,
)

model_df[available_predictors] = model_df[available_predictors].apply(pd.to_numeric, errors="coerce")
model_df[available_predictors] = model_df[available_predictors].fillna(
    model_df[available_predictors].median(numeric_only=True)
)

# Ensure binary outcome (0/1 only)
model_df["aan_persistent_followup"] = (
    pd.to_numeric(model_df["aan_persistent_followup"], errors="coerce")
    .fillna(0)
    .clip(0, 1)
    .astype(int)
)

X = pd.concat([model_df[available_predictors], interval_dummies], axis=1)
y = model_df["aan_persistent_followup"].to_numpy()
groups = model_df[ID_COL].to_numpy()

print(f"Modeling rows (pooled intervals): {model_df.shape[0]}")
print(f"Predictors used ({len(available_predictors)}): {available_predictors}")
print("Interval dummy columns:", list(interval_dummies.columns))
print("Group labels for within-person clustering set to ID.")


Modeling rows (pooled intervals): 70
Predictors used (14): ['baseline_tii', 'baseline_bs', 'baseline_dres', 'baseline_socf', 'baseline_dep', 'baseline_bmi', 'baseline_BE', 'baseline_CB', 'baseline_WSO', 'baseline_FEAR', 'baseline_FAT', 'baseline_LEB', 'cond_bp', 'cond_hw']
Interval dummy columns: ['interval_4_to_5', 'interval_5_to_6']
Group labels for within-person clustering set to ID.


In [66]:
# Repeated holdout (group-aware) to avoid single-class folds
from sklearn.model_selection import GroupShuffleSplit

unique_groups = pd.Series(groups).nunique()
scoring = {
    "precision": make_scorer(precision_score, zero_division=0),
    "recall": make_scorer(recall_score, zero_division=0),
    "bal_acc": make_scorer(balanced_accuracy_score),
}

def summarize_cv(cv_results: dict) -> dict:
    summary = {}
    for metric in scoring:
        scores = cv_results[f"test_{metric}"]
        summary[f"{metric}_mean"] = float(np.mean(scores))
        summary[f"{metric}_sd"] = float(np.std(scores, ddof=1))
    return summary

def run_group_holdout(
    estimator,
    X,
    y,
    groups,
    n_splits: int = 40,
    test_size: float = 0.25,
    random_state: int = 42,
):
    splitter = GroupShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    valid_splits = []
    for train_idx, test_idx in splitter.split(X, y, groups):
        # Skip splits where the test set has only one class
        if pd.Series(y[test_idx]).nunique() < 2:
            continue
        valid_splits.append((train_idx, test_idx))
    if not valid_splits:
        raise ValueError("No holdout splits with both classes; cannot score.")
    cv_results = cross_validate(
        estimator,
        X,
        y,
        cv=valid_splits,
        groups=groups,
        scoring=scoring,
    )
    return cv_results, len(valid_splits)


In [67]:
# Diagnostics: outcome balance and holdout fold composition
print("Outcome class counts (overall):")
print(pd.Series(y).value_counts().rename("count"))

print()
print("Outcome by interval_label:")
print(
    model_df.groupby("interval_label")["aan_persistent_followup"]
    .value_counts()
    .unstack(fill_value=0)
    .rename(columns={0: "remission", 1: "persistence"})
)

if pd.Series(y).nunique() < 2:
    print("WARNING: Only one class present overall; precision/recall will be undefined.")

diag_splitter = GroupShuffleSplit(n_splits=20, test_size=0.25, random_state=42)
fold_rows = []
for split_idx, (train_idx, test_idx) in enumerate(diag_splitter.split(X, y, groups)):
    test_classes = sorted(pd.Series(y[test_idx]).unique().tolist())
    test_counts = pd.Series(y[test_idx]).value_counts().to_dict()
    fold_rows.append(
        {
            "fold": split_idx,
            "test_classes": test_classes,
            "test_counts": test_counts,
        }
    )
fold_df = pd.DataFrame(fold_rows)
print()
print("Holdout fold class composition (test sets):")
display(fold_df[["fold", "test_classes", "test_counts"]])

single_class_folds = [row for row in fold_rows if len(row["test_classes"]) < 2]
if single_class_folds:
    print("WARNING: Some test folds contain a single class; precision/recall warnings may occur.")


Outcome class counts (overall):
0    52
1    18
Name: count, dtype: int64

Outcome by interval_label:
aan_persistent_followup  remission  persistence
interval_label                                 
1_to_4                          26            7
4_to_5                          21           10
5_to_6                           5            1

Holdout fold class composition (test sets):


,fold,test_classes,test_counts
0,0,"[0, 1]","{0: 15, 1: 3}"
1,1,"[0, 1]","{0: 15, 1: 3}"
2,2,"[0, 1]","{0: 14, 1: 4}"
3,3,"[0, 1]","{0: 12, 1: 6}"
4,4,"[0, 1]","{0: 17, 1: 1}"
5,5,"[0, 1]","{0: 14, 1: 4}"
6,6,"[0, 1]","{0: 15, 1: 3}"
7,7,"[0, 1]","{0: 13, 1: 5}"
8,8,"[0, 1]","{0: 13, 1: 5}"
9,9,"[0, 1]","{0: 12, 1: 6}"


In [72]:
# ANOVA-style contrasts (persistence vs remission) on baseline predictors with FDR correction
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

# Enforce a specific construct output order
_CONSTRUCT_LABELS = {
    "baseline_bs": "Body dissatisfaction",
    "baseline_dep": "Negative affect",
    "baseline_dres": "Dieting",
    "baseline_bmi": "BMI",
    "baseline_socf": "Psychosocial functioning",
    "baseline_tii": "Thin-ideal internalization",
    "baseline_BE": "Binge eating",
    "baseline_CB": "Compensatory behaviors",
    "baseline_FAT": "Feeling fat",
    "baseline_FEAR": "Fear of weight gain",
    "baseline_LEB": "Lower-than-expected BMI",
    "baseline_WSO": "Weight/shape overvaluation",
    "cond_bp": "Study condition: BP",
    "cond_hw": "Study condition: HW",
}
_DESIRED_PREDICTOR_ORDER = [
    "baseline_bs",
    "baseline_dep",
    "baseline_dres",
    "baseline_bmi",
    "baseline_socf",
    "baseline_tii",
    "baseline_BE",
    "baseline_CB",
    "baseline_FAT",
    "baseline_FEAR",
    "baseline_LEB",
    "baseline_WSO",
]


def _anova_effect_stats(aov: pd.DataFrame, term: str) -> dict:
    """Return F, p, eta^2, partial eta^2 for a term from statsmodels ANOVA table."""
    if term not in aov.index:
        return {"F": np.nan, "p": np.nan, "eta2": np.nan, "eta2_partial": np.nan}
    F = float(aov.loc[term, "F"]) if "F" in aov.columns else np.nan
    p_val = float(aov.loc[term, "PR(>F)"]) if "PR(>F)" in aov.columns else np.nan
    if "sum_sq" not in aov.columns:
        return {"F": F, "p": p_val, "eta2": np.nan, "eta2_partial": np.nan}
    ss_term = float(aov.loc[term, "sum_sq"])
    ss_total = float(aov["sum_sq"].sum())
    ss_resid = float(aov.loc["Residual", "sum_sq"]) if "Residual" in aov.index else np.nan
    eta2 = ss_term / ss_total if ss_total else np.nan
    denom = ss_term + ss_resid if (ss_term is not None and ss_resid is not None) else np.nan
    eta2_partial = ss_term / denom if denom else np.nan
    return {"F": F, "p": p_val, "eta2": eta2, "eta2_partial": eta2_partial}


condition_covariates = [c for c in ["cond_bp", "cond_hw"] if c in model_df.columns]
# Include study conditions in the tested predictors as well
anova_predictors = [p for p in available_predictors]

anova_rows = []
for pred in anova_predictors:
    covariates_for_pred = [c for c in condition_covariates if c != pred]

    data_dict = {
        "y": pd.to_numeric(model_df[pred], errors="coerce"),
        "group": model_df["aan_persistent_followup"].astype(int),
    }
    for cov in covariates_for_pred:
        data_dict[cov] = pd.to_numeric(model_df[cov], errors="coerce")
    data = pd.DataFrame(data_dict).dropna()
    if data.empty or data["group"].nunique() < 2:
        continue

    formula = "y ~ C(group)"
    if covariates_for_pred:
        formula += " + " + " + ".join(covariates_for_pred)

    model = smf.ols(formula, data=data).fit()
    aov = sm.stats.anova_lm(model, typ=2)

    group_stats = _anova_effect_stats(aov, "C(group)")
    means = data.groupby("group")["y"].agg(["mean", "std", "count"])
    n0, n1 = means.loc[0, "count"], means.loc[1, "count"]
    sd0, sd1 = means.loc[0, "std"], means.loc[1, "std"]
    pooled_sd = (
        np.sqrt(((n0 - 1) * sd0**2 + (n1 - 1) * sd1**2) / (n0 + n1 - 2))
        if (n0 + n1 - 2) > 0
        else np.nan
    )
    cohens_d = (
        (means.loc[1, "mean"] - means.loc[0, "mean"]) / pooled_sd
        if pooled_sd and not np.isnan(pooled_sd)
        else np.nan
    )

    anova_rows.append(
        {
            "predictor": pred,
            "construct": _CONSTRUCT_LABELS.get(pred, pred),
            "mean_remission": float(means.loc[0, "mean"]),
            "sd_remission": float(means.loc[0, "std"]),
            "n_remission": int(n0),
            "mean_persistence": float(means.loc[1, "mean"]),
            "sd_persistence": float(means.loc[1, "std"]),
            "n_persistence": int(n1),
            "F": group_stats["F"],
            "p": group_stats["p"],
            "eta2": group_stats["eta2"],
            "cohens_d": cohens_d,
        }
    )

if anova_rows:
    anova_df = pd.DataFrame(anova_rows)

    # BH-FDR correction across all tested predictors
    anova_df["q_fdr"] = multipletests(anova_df["p"].astype(float), method="fdr_bh")[1]

    # Enforce requested construct ordering (extras, if any, go at the end)
    extras = sorted([p for p in anova_df["predictor"].unique().tolist() if p not in _DESIRED_PREDICTOR_ORDER])
    ordered_categories = _DESIRED_PREDICTOR_ORDER + extras
    anova_df["predictor"] = pd.Categorical(
        anova_df["predictor"], categories=ordered_categories, ordered=True
    )
    anova_df = anova_df.sort_values("predictor", kind="stable")

    print("ANOVA/ANCOVA results (adjusted for condition covariates; conditions included as tested predictors; BH-FDR applied):")
    cols = [
        "construct",
        "mean_remission",
        "sd_remission",
        "n_remission",
        "mean_persistence",
        "sd_persistence",
        "n_persistence",
        "F",
        "p",
        "q_fdr",
        "eta2",
        "cohens_d",
    ]
    out = anova_df[cols].copy()
    round_cols = [
        "mean_remission",
        "sd_remission",
        "mean_persistence",
        "sd_persistence",
        "F",
        "p",
        "q_fdr",
        "eta2",
        "cohens_d",
    ]
    out[round_cols] = out[round_cols].apply(pd.to_numeric, errors="coerce").round(3)
    display(out)
else:
    print("No predictors had data for both groups; ANOVA not run.")

ANOVA/ANCOVA results (adjusted for condition covariates; conditions included as tested predictors; BH-FDR applied):


,construct,mean_remission,sd_remission,n_remission,mean_persistence,sd_persistence,n_persistence,F,p,q_fdr,eta2,cohens_d
1,Body dissatisfaction,3.642,0.890,52,3.844,0.984,18,0.594,0.444,0.850,0.009,0.221
4,Negative affect,0.521,0.992,52,0.693,0.827,18,0.431,0.514,0.850,0.006,0.181
2,Dieting,3.234,0.826,52,3.305,0.801,18,0.084,0.773,0.902,0.001,0.086
5,BMI,24.150,3.984,52,26.038,7.360,18,1.757,0.190,0.850,0.025,0.374
3,Psychosocial functioning,2.283,0.643,52,2.397,0.465,18,0.409,0.524,0.850,0.006,0.189
0,Thin-ideal internalization,3.854,0.420,52,3.935,0.406,18,0.473,0.494,0.850,0.007,0.195
6,Binge eating,0.933,3.003,52,1.222,2.211,18,0.160,0.691,0.879,0.002,0.102
7,Compensatory behaviors,4.500,6.737,52,6.611,9.331,18,0.945,0.335,0.850,0.013,0.283
10,Feeling fat,4.115,2.102,52,4.167,2.407,18,0.003,0.954,0.954,0.000,0.024
9,Fear of weight gain,5.154,0.916,52,5.667,0.686,18,4.619,0.035,0.247,0.058,0.593


In [73]:
# ANOVA-style contrast on parental education (pared = mean of w1paed and w1maed)
parent_ed_cols = [c for c in ["w1paed", "w1maed"] if c in model_df.columns]
covariates = [c for c in ["cond_bp", "cond_hw"] if c in model_df.columns]

if not parent_ed_cols:
    print("w1paed/w1maed not present; pared analysis not run.")
else:
    pared_series = model_df[parent_ed_cols].apply(pd.to_numeric, errors="coerce").mean(axis=1)
    model_df["pared"] = pared_series
    pared_data = pd.DataFrame({
        "pared": pared_series,
        "group": model_df["aan_persistent_followup"].astype(int),
    })
    for cov in covariates:
        pared_data[cov] = pd.to_numeric(model_df[cov], errors="coerce")
    pared_data = pared_data.dropna()
    if pared_data.empty or pared_data["group"].nunique() < 2:
        print("Not enough pared data across both groups to run ANOVA.")
    else:
        formula = "pared ~ C(group)"
        if covariates:
            formula += " + " + " + ".join(covariates)
        pared_model = smf.ols(formula, data=pared_data).fit()
        pared_aov = sm.stats.anova_lm(pared_model, typ=2)

        group_stats = _anova_effect_stats(pared_aov, "C(group)")
        means = pared_data.groupby("group")["pared"].agg(["mean", "std", "count"])
        n0, n1 = means.loc[0, "count"], means.loc[1, "count"]
        sd0, sd1 = means.loc[0, "std"], means.loc[1, "std"]
        pooled_sd = (
            np.sqrt(((n0 - 1) * sd0 ** 2 + (n1 - 1) * sd1 ** 2) / (n0 + n1 - 2))
            if (n0 + n1 - 2) > 0
            else np.nan
        )
        cohens_d = (
            (means.loc[1, "mean"] - means.loc[0, "mean"]) / pooled_sd
            if pooled_sd and not np.isnan(pooled_sd)
            else np.nan
        )

        pared_out = {
            "mean_remission": means.loc[0, "mean"],
            "sd_remission": means.loc[0, "std"],
            "n_remission": int(n0),
            "mean_persistence": means.loc[1, "mean"],
            "sd_persistence": means.loc[1, "std"],
            "n_persistence": int(n1),
            "F_group": group_stats["F"],
            "p_group": group_stats["p"],
            "eta2_group": group_stats["eta2"],
            "eta2p_group": group_stats["eta2_partial"],
            "cohens_d": cohens_d,
        }
        for cov in covariates:
            cov_stats = _anova_effect_stats(pared_aov, cov)
            pared_out[f"F_{cov}"] = cov_stats["F"]
            pared_out[f"p_{cov}"] = cov_stats["p"]
            pared_out[f"eta2_{cov}"] = cov_stats["eta2"]
            pared_out[f"eta2p_{cov}"] = cov_stats["eta2_partial"]

        print("Parental education (pared = mean of w1paed & w1maed): ANCOVA adjusted for condition, includes eta^2")
        display(pd.DataFrame([pared_out]))

w1paed/w1maed not present; pared analysis not run.


In [74]:
# Expanded grid search for standardized logistic regression (optimize balanced accuracy)
# Exploratory settings: keep this fast (small grid, fewer CV splits)
from sklearn.impute import SimpleImputer

multi_X = pd.concat([model_df[available_predictors], interval_dummies], axis=1)

cond_cols = [c for c in ["cond_bp", "cond_hw"] if c in multi_X.columns]
print("Study condition columns included in X:", cond_cols)

logreg_pipe = Pipeline(
    [
        ("impute", SimpleImputer(strategy="median")),
        ("scale", StandardScaler()),
        (
            "clf",
            LogisticRegression(
                max_iter=8000,
                class_weight="balanced",
                random_state=42,
            ),
        ),
    ]
)

# Small-but-broader grid for exploratory analysis
c_grid = [0.01, 0.1, 1.0, 10.0]
param_grid = [
    {
        "clf__solver": ["liblinear"],
        "clf__penalty": ["l2"],
        "clf__C": c_grid,
    },
    {
        "clf__solver": ["liblinear"],
        "clf__penalty": ["l1"],
        "clf__C": c_grid,
    },
    {
        "clf__solver": ["saga"],
        "clf__penalty": ["elasticnet"],
        "clf__C": [0.01, 0.1, 1.0],
        "clf__l1_ratio": [0.2, 0.5, 0.8],
    },
]

gss = GroupShuffleSplit(n_splits=25, test_size=0.25, random_state=42)
logreg_grid = GridSearchCV(
    estimator=logreg_pipe,
    param_grid=param_grid,
    scoring="balanced_accuracy",
    cv=gss,
    n_jobs=-1,
    refit=True,
)

logreg_grid.fit(multi_X, y, groups=groups)
best_logreg = logreg_grid.best_estimator_
print("Best balanced accuracy (GroupShuffleSplit):", logreg_grid.best_score_)
print("Best params:", logreg_grid.best_params_)

best_cv, n_splits_used = run_group_holdout(best_logreg, multi_X, y, groups, n_splits=15)
best_summary = {"model": "logreg_tuned", "splits": n_splits_used, **summarize_cv(best_cv)}
print("Tuned standardized logistic regression (balanced) performance with mean and SD across splits:")
display(pd.DataFrame([best_summary]))

Study condition columns included in X: ['cond_bp', 'cond_hw']
Best balanced accuracy (GroupShuffleSplit): 0.5410140202934319
Best params: {'clf__C': 0.1, 'clf__l1_ratio': 0.5, 'clf__penalty': 'elasticnet', 'clf__solver': 'saga'}
Tuned standardized logistic regression (balanced) performance with mean and SD across splits:


,model,splits,precision_mean,precision_sd,recall_mean,recall_sd,bal_acc_mean,bal_acc_sd
0,logreg_tuned,15,0.24135,0.151848,0.445714,0.335238,0.493935,0.165607


In [75]:
# Leave-one-out (group) and threshold tuning helpers
loo_splitter = LeaveOneGroupOut()

def collect_loo_probabilities(estimator, X, y, groups):
    y_true_all = []
    y_score_all = []
    splits = 0
    for train_idx, test_idx in loo_splitter.split(X, y, groups):
        if pd.Series(y[train_idx]).nunique() < 2:
            # Skip if training set has only one class
            continue
        est = clone(estimator)
        est.fit(X[train_idx], y[train_idx])
        proba = est.predict_proba(X[test_idx])[:, 1]
        y_true_all.append(y[test_idx])
        y_score_all.append(proba)
        splits += 1
    if not y_true_all:
        raise ValueError("No LOO splits with both classes in train; cannot compute predictions.")
    return np.concatenate(y_true_all), np.concatenate(y_score_all), splits

def threshold_grid(y_true, y_score, thresholds=None):
    if thresholds is None:
        thresholds = np.linspace(0, 1, 201)
    rows = []
    y_true = np.asarray(y_true).astype(int)
    y_score = np.asarray(y_score).astype(float)
    for thr in thresholds:
        preds = (y_score >= thr).astype(int)
        tp = int(((preds == 1) & (y_true == 1)).sum())
        fp = int(((preds == 1) & (y_true == 0)).sum())
        fn = int(((preds == 0) & (y_true == 1)).sum())
        tn = int(((preds == 0) & (y_true == 0)).sum())
        precision = tp / (tp + fp) if (tp + fp) else 0.0
        sensitivity = tp / (tp + fn) if (tp + fn) else 0.0
        specificity = tn / (tn + fp) if (tn + fp) else 0.0
        bal_acc = (sensitivity + specificity) / 2
        accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) else 0.0
        f1 = 2 * tp / (2 * tp + fp + fn) if (tp + fp + fn) else 0.0
        rows.append(
            {
                "threshold": float(thr),
                "precision": precision,
                "recall": sensitivity,
                "sensitivity": sensitivity,
                "specificity": specificity,
                "balanced_accuracy": bal_acc,
                "accuracy": accuracy,
                "f1": f1,
                "tp": tp,
                "fp": fp,
                "fn": fn,
                "tn": tn,
            }
        )
    return pd.DataFrame(rows)

def pick_threshold_balanced_accuracy(
    y_true,
    y_score,
    thresholds=None,
    min_sensitivity=0.53,
    min_specificity=0.53,
    prefer_feasible=True,
):
    """Pick threshold maximizing balanced accuracy, optionally requiring sens/spec minima."""
    df = threshold_grid(y_true, y_score, thresholds=thresholds)
    feasible = df.copy()
    if min_sensitivity is not None:
        feasible = feasible[feasible["sensitivity"] >= float(min_sensitivity)]
    if min_specificity is not None:
        feasible = feasible[feasible["specificity"] >= float(min_specificity)]
    if prefer_feasible and not feasible.empty:
        best = feasible.sort_values(
            ["balanced_accuracy", "sensitivity", "specificity", "threshold"],
            ascending=[False, False, False, True],
        ).iloc[0]
        best = best.copy()
        best["constraint_met"] = True
        return best, df
    best = df.sort_values(
        ["balanced_accuracy", "sensitivity", "specificity", "threshold"],
        ascending=[False, False, False, True],
    ).iloc[0]
    best = best.copy()
    best["constraint_met"] = False if prefer_feasible else True
    return best, df

def pick_threshold_for_precision(threshold_df: pd.DataFrame, target_precision: float = 0.6) -> pd.Series:
    meets = threshold_df[threshold_df["precision"] >= target_precision]
    if meets.empty:
        return threshold_df.sort_values(["precision", "recall"], ascending=[False, False]).iloc[0]
    return meets.sort_values(["precision", "recall", "threshold"], ascending=[True, False, True]).iloc[0]

In [83]:
# Leave-one-group-out evaluation for tuned standardized logistic regression + naive baselines
# Threshold tuned to maximize balanced accuracy; prefers thresholds where sensitivity & specificity >= 0.53 if feasible
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score

multi_X = pd.concat([model_df[available_predictors], interval_dummies], axis=1)
cond_cols = [c for c in ["cond_bp", "cond_hw"] if c in multi_X.columns]
print("Study condition columns included in X:", cond_cols)

X_array = multi_X.to_numpy()

logreg_for_loo = best_logreg if "best_logreg" in locals() else Pipeline(
    [
        ("impute", SimpleImputer(strategy="median")),
        ("scale", StandardScaler()),
        ("clf", LogisticRegression(max_iter=8000, class_weight="balanced", solver="liblinear", random_state=42)),
    ]
)

if len(np.unique(y)) < 2:
    raise ValueError("Outcome has only one class; cannot evaluate.")

y_true_cv, y_score_cv, splits_used = collect_loo_probabilities(logreg_for_loo, X_array, y, groups)
ap = average_precision_score(y_true_cv, y_score_cv) if len(np.unique(y_true_cv)) > 1 else np.nan
auroc = roc_auc_score(y_true_cv, y_score_cv) if len(np.unique(y_true_cv)) > 1 else np.nan

best_row, thr_df = pick_threshold_balanced_accuracy(
    y_true_cv,
    y_score_cv,
    thresholds=np.linspace(0, 1, 201),
    min_sensitivity=0.53,
    min_specificity=0.53,
    prefer_feasible=True,
)

best_thr = float(best_row["threshold"])
precision = float(best_row["precision"])
recall = float(best_row["recall"])
specificity = float(best_row["specificity"])
best_bal = float(best_row["balanced_accuracy"])
accuracy = float(best_row["accuracy"]) if "accuracy" in best_row.index else float(np.mean((y_score_cv >= best_thr) == y_true_cv))
constraint_met = bool(best_row["constraint_met"])
f1 = 2 * precision * recall / (precision + recall + 1e-9) if (precision + recall) else 0.0

rows = [
    {
        "model": "logreg_tuned",
        "splits": splits_used,
        "ap": ap,
        "auroc": auroc,
        "threshold": best_thr,
        "precision": precision,
        "recall": recall,
        "specificity": specificity,
        "bal_acc": best_bal,
        "accuracy": accuracy,
        "f1": f1,
        "sens_spec_ge_0.53": constraint_met,
    }
]

prevalence = float(np.mean(y_true_cv))
naive_scores = [
    ("naive_all_negative", np.zeros_like(y_true_cv), np.zeros_like(y_score_cv, dtype=float), 1.0),
    ("naive_all_positive", np.ones_like(y_true_cv), np.ones_like(y_score_cv, dtype=float), 0.0),
]
for name, pred_labels, score_vals, thr in naive_scores:
    ap_base = average_precision_score(y_true_cv, pred_labels)
    auroc_base = roc_auc_score(y_true_cv, score_vals) if len(np.unique(y_true_cv)) > 1 else np.nan
    tp_b = int(((pred_labels == 1) & (y_true_cv == 1)).sum())
    fp_b = int(((pred_labels == 1) & (y_true_cv == 0)).sum())
    fn_b = int(((pred_labels == 0) & (y_true_cv == 1)).sum())
    tn_b = int(((pred_labels == 0) & (y_true_cv == 0)).sum())
    prec_b = tp_b / (tp_b + fp_b) if (tp_b + fp_b) else 0.0
    rec_b = tp_b / (tp_b + fn_b) if (tp_b + fn_b) else 0.0
    spec_b = tn_b / (tn_b + fp_b) if (tn_b + fp_b) else 0.0
    acc_b = (tp_b + tn_b) / (tp_b + tn_b + fp_b + fn_b) if (tp_b + tn_b + fp_b + fn_b) else 0.0
    f1_b = 2 * prec_b * rec_b / (prec_b + rec_b + 1e-9) if (prec_b + rec_b) else 0.0
    bal_b = (rec_b + spec_b) / 2
    rows.append(
        {
            "model": name,
            "splits": splits_used,
            "ap": ap_base,
            "auroc": auroc_base,
            "threshold": thr,
            "precision": prec_b,
            "recall": rec_b,
            "specificity": spec_b,
            "bal_acc": bal_b,
            "accuracy": acc_b,
            "f1": f1_b,
            "sens_spec_ge_0.53": (rec_b >= 0.53) and (spec_b >= 0.53),
        }
    )

rows.append(
    {
        "model": "naive_random_prevalence_ap",
        "splits": splits_used,
        "ap": prevalence,
        "auroc": 0.5,
        "threshold": np.nan,
        "precision": prevalence,
        "recall": 1.0,
        "specificity": 0.0,
        "bal_acc": 0.5,
        "accuracy": np.nan,
        "f1": 2 * prevalence / (1 + prevalence) if prevalence + 1 else 0.0,
        "sens_spec_ge_0.53": False,
    }
)

loo_results = pd.DataFrame(rows)
if not constraint_met:
    print("NOTE: No threshold achieved BOTH sensitivity and specificity >= 0.53; showing best balanced-accuracy threshold instead.")
print("LOO tuned logistic regression vs naive baselines; threshold chosen to maximize balanced accuracy (prefers sens/spec >= 0.53):")
display(loo_results)

Study condition columns included in X: ['cond_bp', 'cond_hw']
LOO tuned logistic regression vs naive baselines; threshold chosen to maximize balanced accuracy (prefers sens/spec >= 0.53):


,model,splits,ap,auroc,threshold,precision,recall,specificity,bal_acc,accuracy,f1,sens_spec_ge_0.53
0,logreg_tuned,70,0.326618,0.586538,0.5,0.400000,0.666667,0.653846,0.660256,0.657143,0.500000,True
1,naive_all_negative,70,0.257143,0.500000,1.0,0.000000,0.000000,1.000000,0.500000,0.742857,0.000000,False
2,naive_all_positive,70,0.257143,0.500000,0.0,0.257143,1.000000,0.000000,0.500000,0.257143,0.409091,False
3,naive_random_prevalence_ap,70,0.257143,0.500000,NaN,0.257143,1.000000,0.000000,0.500000,NaN,0.409091,False


In [82]:
# Grid search tuned for balanced accuracy (sensitivity/specificity balance)
# Exploratory settings: keep this fast (small grid, fewer CV splits)
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score

multi_X = pd.concat([model_df[available_predictors], interval_dummies], axis=1)
cond_cols = [c for c in ["cond_bp", "cond_hw"] if c in multi_X.columns]
print("Study condition columns included in X:", cond_cols)

logreg_pipe = Pipeline(
    [
        ("impute", SimpleImputer(strategy="median")),
        ("scale", StandardScaler()),
        (
            "clf",
            LogisticRegression(
                max_iter=8000,
                class_weight="balanced",
                random_state=42,
            ),
        ),
    ]
)

c_grid = [0.01, 0.1, 1.0, 10.0]
param_grid = [
    {
        "clf__solver": ["liblinear"],
        "clf__penalty": ["l2"],
        "clf__C": c_grid,
    },
    {
        "clf__solver": ["liblinear"],
        "clf__penalty": ["l1"],
        "clf__C": c_grid,
    },
    {
        "clf__solver": ["saga"],
        "clf__penalty": ["elasticnet"],
        "clf__C": [0.01, 0.1, 1.0],
        "clf__l1_ratio": [0.2, 0.5, 0.8],
    },
]

gss = GroupShuffleSplit(n_splits=15, test_size=0.25, random_state=42)
grid_balacc = GridSearchCV(
    estimator=logreg_pipe,
    param_grid=param_grid,
    scoring="balanced_accuracy",
    cv=gss,
    n_jobs=-1,
    refit=True,
)

grid_balacc.fit(multi_X, y, groups=groups)
best_balacc_model = grid_balacc.best_estimator_
print("Best balanced accuracy (GroupShuffleSplit):", grid_balacc.best_score_)
print("Best params (balanced accuracy grid):", grid_balacc.best_params_)

# Mean/SD of metrics across valid group holdout splits
cv_bal, splits_bal = run_group_holdout(best_balacc_model, multi_X, y, groups, n_splits=15)
bal_summary = {"model": "logreg_balacc_tuned", "splits": splits_bal, **summarize_cv(cv_bal)}
print("Balanced-accuracy tuned logistic regression (mean/SD across splits):")
display(pd.DataFrame([bal_summary]))

# Threshold tuning via LOO to maximize balanced accuracy (prefers sens/spec >= 0.53 if feasible)
if len(np.unique(y)) < 2:
    raise ValueError("Outcome has only one class; cannot evaluate.")

y_true_cv, y_score_cv, splits_used = collect_loo_probabilities(
    best_balacc_model, multi_X.to_numpy(), y, groups
)

auroc = roc_auc_score(y_true_cv, y_score_cv) if len(np.unique(y_true_cv)) > 1 else np.nan

best_row, thr_df = pick_threshold_balanced_accuracy(
    y_true_cv,
    y_score_cv,
    thresholds=np.linspace(0, 1, 201),
    min_sensitivity=0.53,
    min_specificity=0.53,
    prefer_feasible=True,
)

best_thr = float(best_row["threshold"])
recall = float(best_row["recall"])
specificity = float(best_row["specificity"])
best_bal = float(best_row["balanced_accuracy"])
accuracy = float(best_row["accuracy"]) if "accuracy" in best_row.index else float(np.mean((y_score_cv >= best_thr) == y_true_cv))
precision = float(best_row["precision"])
f1 = 2 * precision * recall / (precision + recall + 1e-9) if (precision + recall) else 0.0
constraint_met = bool(best_row["constraint_met"])

if not constraint_met:
    print("NOTE: No threshold achieved BOTH sensitivity and specificity >= 0.53; showing best balanced-accuracy threshold instead.")

print("LOO tuned for balanced accuracy: threshold, sensitivity, specificity, accuracy, AUROC, precision, F1")
display(
    pd.DataFrame(
        [
            {
                "model": "logreg_balacc_tuned",
                "splits": splits_used,
                "threshold": best_thr,
                "recall": recall,
                "specificity": specificity,
                "bal_acc": best_bal,
                "accuracy": accuracy,
                "auroc": auroc,
                "precision": precision,
                "f1": f1,
                "sens_spec_ge_0.53": constraint_met,
            }
        ]
    )
)

Study condition columns included in X: ['cond_bp', 'cond_hw']
Best balanced accuracy (GroupShuffleSplit): 0.5
Best params (balanced accuracy grid): {'clf__C': 0.01, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
Balanced-accuracy tuned logistic regression (mean/SD across splits):


,model,splits,precision_mean,precision_sd,recall_mean,recall_sd,bal_acc_mean,bal_acc_sd
0,logreg_balacc_tuned,15,0.0,0.0,0.0,0.0,0.5,0.0


NOTE: No threshold achieved BOTH sensitivity and specificity >= 0.53; showing best balanced-accuracy threshold instead.
LOO tuned for balanced accuracy: threshold, sensitivity, specificity, accuracy, AUROC, precision, F1


,model,splits,threshold,recall,specificity,bal_acc,accuracy,auroc,precision,f1,sens_spec_ge_0.53
0,logreg_balacc_tuned,70,0.0,1.0,0.0,0.5,0.257143,0.5,0.257143,0.409091,False


In [49]:
# Bootstrap CIs on LOO predictions to strengthen reporting
# Requires y_true_cv and y_score_cv from the preceding LOO block
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, average_precision_score

if 'y_true_cv' not in locals() or 'y_score_cv' not in locals():
    raise RuntimeError("Run the LOO evaluation cell first to populate y_true_cv and y_score_cv.")

rng = np.random.default_rng(123)
# Exploratory: fewer resamples to keep runtime reasonable
B = 1000
rows = []

thr_val = float(best_thr) if 'best_thr' in locals() else 0.5
for _ in range(B):
    idx = rng.integers(0, len(y_true_cv), size=len(y_true_cv))
    yb_true = y_true_cv[idx]
    yb_score = y_score_cv[idx]
    preds = (yb_score >= thr_val).astype(int)

    tn = int(((preds == 0) & (yb_true == 0)).sum())
    fp = int(((preds == 1) & (yb_true == 0)).sum())
    spec = tn / (tn + fp) if (tn + fp) else 0.0

    bal_acc = balanced_accuracy_score(yb_true, preds) if len(np.unique(yb_true)) > 1 else np.nan
    sens = recall_score(yb_true, preds, zero_division=0)
    prec = precision_score(yb_true, preds, zero_division=0)
    ap = average_precision_score(yb_true, yb_score) if len(np.unique(yb_true)) > 1 else np.nan
    rows.append({"balanced_accuracy": bal_acc, "sensitivity": sens, "specificity": spec, "precision": prec, "ap": ap})

boot = pd.DataFrame(rows)
ci = boot.quantile([0.025, 0.5, 0.975]).T.rename(columns={0.025: "ci_low", 0.5: "median", 0.975: "ci_high"})
print(f"Bootstrap CIs ({B} resamples) for LOO predictions at the selected threshold:")
display(ci)

Bootstrap CIs (1000 resamples) for LOO predictions at the selected threshold:


,ci_low,median,ci_high
balanced_accuracy,0.465886,0.585000,0.679952
sensitivity,0.625000,0.844130,1.000000
specificity,0.203980,0.326923,0.454545
precision,0.180000,0.300000,0.433962
ap,0.170237,0.306585,0.499869


In [50]:
# Logistic and Balanced RF grid search (AUROC), then threshold tuning for balanced accuracy
# Exploratory settings: fewer CV splits and smaller grids to keep runtime reasonable
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score

multi_X = pd.concat([model_df[available_predictors], interval_dummies], axis=1)
multi_X = multi_X.fillna(multi_X.median(numeric_only=True))
X_array = multi_X.to_numpy()

# Common CV splitter (group-aware)
gss = GroupShuffleSplit(n_splits=15, test_size=0.25, random_state=42)

def build_valid_splits(splitter, X, y, groups):
    valid = []
    for tr, te in splitter.split(X, y, groups):
        y_tr, y_te = y[tr], y[te]
        if pd.Series(y_tr).nunique() < 2 or pd.Series(y_te).nunique() < 2:
            continue
        valid.append((tr, te))
    if not valid:
        raise ValueError("No group splits with both classes in train/test; cannot score AUROC.")
    return valid

cv_splits = build_valid_splits(gss, multi_X, y, groups)
print(f"Valid splits with both classes (train/test): {len(cv_splits)}")

# Pipelines
log_pipe = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(max_iter=3000, class_weight="balanced")),
    ]
)

# Small AUROC grid for exploratory analysis
log_grid = {
    "model__C": [0.01, 0.1, 1.0],
    "model__penalty": ["l2"],
    "model__solver": ["liblinear"],
}

brf_pipe = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("model", BalancedRandomForestClassifier(random_state=42, n_jobs=-1)),
    ]
)

# Small BRF grid for exploratory analysis
brf_grid = {
    "model__n_estimators": [100],
    "model__max_depth": [3, 6],
    "model__min_samples_leaf": [4],
    "model__max_features": ["sqrt"],
}

log_search = GridSearchCV(
    estimator=log_pipe,
    param_grid=log_grid,
    cv=cv_splits,
    scoring="roc_auc",
    n_jobs=-1,
    error_score="raise",
    refit=True,
)

brf_search = GridSearchCV(
    estimator=brf_pipe,
    param_grid=brf_grid,
    cv=cv_splits,
    scoring="roc_auc",
    n_jobs=-1,
    error_score="raise",
    refit=True,
)

log_search.fit(multi_X, y)
brf_search.fit(multi_X, y)

log_best = log_search.best_estimator_
brf_best = brf_search.best_estimator_

print("Logistic (AUROC) best score:", log_search.best_score_)
print("Logistic best params:", log_search.best_params_)
print("Balanced RF (AUROC) best score:", brf_search.best_score_)
print("Balanced RF best params:", brf_search.best_params_)

def tune_balanced_accuracy(estimator, X_vals, y_true, groups_arr, label: str):
    y_true_cv, y_score_cv, splits_used = collect_loo_probabilities(estimator, X_vals, y_true, groups_arr)
    best, df_thr = pick_threshold_balanced_accuracy(
        y_true_cv,
        y_score_cv,
        thresholds=np.linspace(0, 1, 201),
        min_sensitivity=0.53,
        min_specificity=0.53,
        prefer_feasible=True,
)
    summary = best.to_dict()
    summary.update({"model": label, "splits": splits_used})
    return summary, df_thr

log_summary, log_thr_df = tune_balanced_accuracy(log_best, X_array, y, groups, "logistic")
brf_summary, brf_thr_df = tune_balanced_accuracy(brf_best, X_array, y, groups, "balanced_rf")

print("Thresholds tuned for balanced accuracy (LOO over groups):")
display(pd.DataFrame([log_summary, brf_summary]))
print("Top thresholds (balanced accuracy) - logistic")
display(log_thr_df.sort_values("balanced_accuracy", ascending=False).head(10))
print("Top thresholds (balanced accuracy) - balanced RF")
display(brf_thr_df.sort_values("balanced_accuracy", ascending=False).head(10))

Valid splits with both classes (train/test): 15
Logistic (AUROC) best score: 0.5232976282976284
Logistic best params: {'model__C': 0.01, 'model__penalty': 'l2', 'model__solver': 'liblinear'}
Balanced RF (AUROC) best score: 0.45264543735131973
Balanced RF best params: {'model__max_depth': 6, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 4, 'model__n_estimators': 100}
Thresholds tuned for balanced accuracy (LOO over groups):


,threshold,precision,recall,sensitivity,specificity,balanced_accuracy,accuracy,f1,tp,fp,fn,tn,constraint_met,model,splits
0,0.470,0.300000,0.833333,0.833333,0.326923,0.580128,0.457143,0.441176,15.0,35.0,3.0,17.0,False,logistic,70
1,0.355,0.283019,0.833333,0.833333,0.269231,0.551282,0.414286,0.422535,15.0,38.0,3.0,14.0,False,balanced_rf,70


Top thresholds (balanced accuracy) - logistic


,threshold,precision,recall,sensitivity,specificity,balanced_accuracy,accuracy,f1,tp,fp,fn,tn
94,0.470,0.300000,0.833333,0.833333,0.326923,0.580128,0.457143,0.441176,15,35,3,17
93,0.465,0.296296,0.888889,0.888889,0.269231,0.579060,0.428571,0.444444,16,38,2,14
91,0.455,0.288136,0.944444,0.944444,0.192308,0.568376,0.385714,0.441558,17,42,1,10
92,0.460,0.288136,0.944444,0.944444,0.192308,0.568376,0.385714,0.441558,17,42,1,10
95,0.475,0.291667,0.777778,0.777778,0.346154,0.561966,0.457143,0.424242,14,34,4,18
96,0.480,0.288889,0.722222,0.722222,0.384615,0.553419,0.471429,0.412698,13,32,5,20
109,0.545,0.333333,0.222222,0.222222,0.846154,0.534188,0.685714,0.266667,4,8,14,44
108,0.540,0.333333,0.222222,0.222222,0.846154,0.534188,0.685714,0.266667,4,8,14,44
90,0.450,0.269841,0.944444,0.944444,0.115385,0.529915,0.328571,0.419753,17,46,1,6
115,0.575,0.400000,0.111111,0.111111,0.942308,0.526709,0.728571,0.173913,2,3,16,49


Top thresholds (balanced accuracy) - balanced RF


,threshold,precision,recall,sensitivity,specificity,balanced_accuracy,accuracy,f1,tp,fp,fn,tn
72,0.360,0.283019,0.833333,0.833333,0.269231,0.551282,0.414286,0.422535,15,38,3,14
71,0.355,0.283019,0.833333,0.833333,0.269231,0.551282,0.414286,0.422535,15,38,3,14
123,0.615,0.500000,0.111111,0.111111,0.961538,0.536325,0.742857,0.181818,2,2,16,50
73,0.365,0.274510,0.777778,0.777778,0.288462,0.533120,0.414286,0.405797,14,37,4,15
74,0.370,0.274510,0.777778,0.777778,0.288462,0.533120,0.414286,0.405797,14,37,4,15
85,0.425,0.275000,0.611111,0.611111,0.442308,0.526709,0.485714,0.379310,11,29,7,23
122,0.610,0.400000,0.111111,0.111111,0.942308,0.526709,0.728571,0.173913,2,3,16,49
125,0.625,0.500000,0.055556,0.055556,0.980769,0.518162,0.742857,0.100000,1,1,17,51
124,0.620,0.500000,0.055556,0.055556,0.980769,0.518162,0.742857,0.100000,1,1,17,51
84,0.420,0.268293,0.611111,0.611111,0.423077,0.517094,0.471429,0.372881,11,30,7,22


In [61]:
# Univariate standardized logit with OR, 95% CI, SE, p, and AUROC (exploratory / in-sample)
from statsmodels.stats.multitest import multipletests
from sklearn.metrics import roc_auc_score

# Enforce a specific construct output order
_CONSTRUCT_LABELS = {
    "baseline_bs": "Body dissatisfaction",
    "baseline_dep": "Negative affect",
    "baseline_dres": "Dieting",
    "baseline_bmi": "BMI",
    "baseline_socf": "Psychosocial functioning",
    "baseline_tii": "Thin-ideal internalization",
    "baseline_BE": "Binge eating",
    "baseline_CB": "Compensatory behaviors",
    "baseline_FAT": "Feeling fat",
    "baseline_FEAR": "Fear of weight gain",
    "baseline_LEB": "Lower-than-expected BMI",
    "baseline_WSO": "Weight/shape overvaluation",
    "cond_bp": "Study condition: BP",
    "cond_hw": "Study condition: HW",
}
_DESIRED_PREDICTOR_ORDER = [
    "baseline_bs",
    "baseline_dep",
    "baseline_dres",
    "baseline_bmi",
    "baseline_socf",
    "baseline_tii",
    "baseline_BE",
    "baseline_CB",
    "baseline_FAT",
    "baseline_FEAR",
    "baseline_LEB",
    "baseline_WSO",
]

condition_covariates = [c for c in ["cond_bp", "cond_hw"] if c in model_df.columns]
# Include study conditions in the tested predictors as well
logit_predictors = [p for p in available_predictors]

uni_rows = []
for pred in logit_predictors:
    covariates_for_pred = [c for c in condition_covariates if c != pred]

    Xi = pd.DataFrame(
        StandardScaler().fit_transform(model_df[[pred]]),
        columns=[pred],
        index=model_df.index,
    )

    extra_blocks = [interval_dummies]
    if covariates_for_pred:
        extra_blocks.append(model_df[covariates_for_pred])

    Xi = pd.concat([Xi] + extra_blocks, axis=1)
    Xi = sm.add_constant(Xi, has_constant="add")

    Xi_num = Xi.apply(pd.to_numeric, errors="coerce")
    row_mask = Xi_num.notna().all(axis=1)
    Xi_clean = Xi_num.loc[row_mask].astype(float)
    y_clean = (
        pd.to_numeric(model_df.loc[Xi_num.index, "aan_persistent_followup"], errors="coerce")
        .loc[row_mask]
        .astype(int)
        .values
    )
    if pd.Series(y_clean).nunique() < 2:
        continue

    try:
        res = sm.Logit(y_clean, Xi_clean).fit(disp=False)
        ci = res.conf_int()
        probs = res.predict(Xi_clean)
        auroc = roc_auc_score(y_clean, probs) if len(np.unique(y_clean)) > 1 else np.nan
        uni_rows.append(
            {
                "predictor": pred,
                "construct": _CONSTRUCT_LABELS.get(pred, pred),
                "n_used": int(len(y_clean)),
                "auroc": float(auroc),
                "se": float(res.bse[pred]),
                "p": float(res.pvalues[pred]),
                "OR": float(np.exp(res.params[pred])),
                "CI_low": float(np.exp(ci.loc[pred, 0])),
                "CI_high": float(np.exp(ci.loc[pred, 1])),
            }
        )
    except Exception as exc:
        print(f"{pred} failed: {exc}")

if uni_rows:
    uni_df = pd.DataFrame(uni_rows)

    # Enforce requested construct ordering (extras, if any, go at the end)
    extras = sorted([p for p in uni_df["predictor"].unique().tolist() if p not in _DESIRED_PREDICTOR_ORDER])
    ordered_categories = _DESIRED_PREDICTOR_ORDER + extras
    uni_df["predictor"] = pd.Categorical(uni_df["predictor"], categories=ordered_categories, ordered=True)
    uni_df = uni_df.sort_values("predictor", kind="stable")

    # (Optional) BH-FDR computed but not displayed in minimal table
    _q = multipletests(uni_df["p"], method="fdr_bh")[1]

    print("Logistic regression (controls: interval + condition; conditions included as tested predictors); AUROC is in-sample (exploratory):")
    cols = ["construct", "auroc", "OR", "CI_low", "CI_high", "se", "p", "n_used"]
    out = uni_df[cols].copy()
    round_cols = ["auroc", "OR", "CI_low", "CI_high", "se", "p"]
    out[round_cols] = out[round_cols].apply(pd.to_numeric, errors="coerce").round(3)
    display(out)
else:
    print("No predictors produced valid univariate models.")

Logistic regression (controls: interval + condition; conditions included as tested predictors); AUROC is in-sample (exploratory):


,construct,auroc,OR,CI_low,CI_high,se,p,n_used
1,Body dissatisfaction,0.647,1.290,0.713,2.333,0.302,0.400,70
4,Negative affect,0.618,1.243,0.721,2.142,0.278,0.434,70
2,Dieting,0.612,1.191,0.675,2.104,0.290,0.546,70
5,BMI,0.631,1.434,0.843,2.442,0.271,0.184,70
3,Psychosocial functioning,0.636,1.353,0.746,2.457,0.304,0.320,70
0,Thin-ideal internalization,0.615,1.295,0.731,2.292,0.291,0.375,70
6,Binge eating,0.599,1.210,0.662,2.209,0.307,0.536,70
7,Compensatory behaviors,0.631,1.345,0.790,2.289,0.271,0.275,70
10,Feeling fat,0.621,1.039,0.600,1.798,0.280,0.892,70
9,Fear of weight gain,0.722,2.291,1.112,4.721,0.369,0.025,70


In [52]:
# Univariate grid searches (AUROC) for each predictor with interval + condition dummies; thresholds tuned for balanced accuracy
# Exploratory settings: fewer CV splits + smaller grids to keep runtime reasonable
from sklearn.model_selection import GroupShuffleSplit
from sklearn.impute import SimpleImputer

univ_results = []
log_uni_searches = {}
brf_uni_searches = {}

# Shared grids (small)
log_grid = {
    "model__C": [0.01, 0.1, 1.0],
    "model__penalty": ["l2"],
    "model__solver": ["liblinear"],
}

brf_grid = {
    "model__n_estimators": [100],
    "model__max_depth": [3],
    "model__min_samples_leaf": [4],
    "model__max_features": ["sqrt"],
}

# CV splitter with filtering to ensure both classes in train/test
uni_gss = GroupShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

def build_valid_splits(splitter, X, y, groups):
    valid = []
    for tr, te in splitter.split(X, y, groups):
        y_tr, y_te = y[tr], y[te]
        if pd.Series(y_tr).nunique() < 2 or pd.Series(y_te).nunique() < 2:
            continue
        valid.append((tr, te))
    if not valid:
        raise ValueError("No group splits with both classes in train/test; cannot score.")
    return valid

uni_cv_splits = build_valid_splits(uni_gss, multi_X, y, groups)
print(f"Valid univariate splits with both classes (train/test): {len(uni_cv_splits)}")

for pred in available_predictors:
    Xi = model_df[[pred]].copy()
    # Add interval and condition dummies (as controls)
    blocks = [Xi, interval_dummies]
    cond_covs = [c for c in ["cond_bp", "cond_hw"] if c in model_df.columns]
    if cond_covs:
        blocks.append(model_df[cond_covs])
    Xi_full = pd.concat(blocks, axis=1)
    Xi_full = Xi_full.apply(pd.to_numeric, errors="coerce")
    Xi_full = Xi_full.fillna(Xi_full.median(numeric_only=True))
    X_vals = Xi_full.to_numpy()

    log_pipe = Pipeline(
        [
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
            ("model", LogisticRegression(max_iter=3000, class_weight="balanced")),
        ]
    )
    log_search = GridSearchCV(
        estimator=log_pipe,
        param_grid=log_grid,
        cv=uni_cv_splits,
        scoring="roc_auc",
        n_jobs=-1,
        error_score="raise",
        refit=True,
)
    log_search.fit(X_vals, y)
    log_uni_searches[pred] = (log_search, X_vals)
    univ_results.append({
        "predictor": pred,
        "model": "logistic",
        "best_auc": log_search.best_score_,
        "best_params": log_search.best_params_,
    })

    brf_pipe = Pipeline(
        [
            ("imputer", SimpleImputer(strategy="median")),
            ("model", BalancedRandomForestClassifier(random_state=42, n_jobs=-1)),
        ]
    )
    brf_search = GridSearchCV(
        estimator=brf_pipe,
        param_grid=brf_grid,
        cv=uni_cv_splits,
        scoring="balanced_accuracy",
        n_jobs=-1,
        error_score="raise",
        refit=True,
)
    brf_search.fit(X_vals, y)
    brf_uni_searches[pred] = (brf_search, X_vals)
    univ_results.append({
        "predictor": pred,
        "model": "balanced_rf",
        "best_auc": brf_search.best_score_,
        "best_params": brf_search.best_params_,
    })

univ_df = pd.DataFrame(univ_results).sort_values("best_auc", ascending=False)
print("Top univariate predictors by score (logistic AUROC, BRF balanced accuracy):")
display(univ_df.head(20))

# Pick best per model
best_log_pred, (best_log_search, best_log_X) = max(log_uni_searches.items(), key=lambda kv: kv[1][0].best_score_)
best_brf_pred, (best_brf_search, best_brf_X) = max(brf_uni_searches.items(), key=lambda kv: kv[1][0].best_score_)
print(f"Best logistic predictor: {best_log_pred} (score {best_log_search.best_score_:.3f})")
print(f"Best balanced RF predictor: {best_brf_pred} (score {best_brf_search.best_score_:.3f})")

# Threshold tuning for balanced accuracy on the best predictors
def tune_bal_acc_uni(label: str, search, X_vals):
    summary, df_thr = tune_balanced_accuracy(search.best_estimator_, X_vals, y, groups, label)
    return summary, df_thr

log_uni_summary, log_uni_thr = tune_bal_acc_uni(f"logistic_{best_log_pred}", best_log_search, best_log_X)
brf_uni_summary, brf_uni_thr = tune_bal_acc_uni(f"balanced_rf_{best_brf_pred}", best_brf_search, best_brf_X)

print("Best-univariate threshold tuning (balanced accuracy):")
display(pd.DataFrame([log_uni_summary, brf_uni_summary]))
print("Top thresholds (logistic best univariate)")
display(log_uni_thr.sort_values("balanced_accuracy", ascending=False).head(10))
print("Top thresholds (balanced RF best univariate)")
display(brf_uni_thr.sort_values("balanced_accuracy", ascending=False).head(10))

Valid univariate splits with both classes (train/test): 10
Top univariate predictors by score (logistic AUROC, BRF balanced accuracy):


,predictor,model,best_auc,best_params
18,baseline_FEAR,logistic,0.602103,"{'model__C': 0.01, 'model__penalty': 'l2', 'mo..."
16,baseline_WSO,logistic,0.573787,"{'model__C': 0.01, 'model__penalty': 'l2', 'mo..."
9,baseline_dep,balanced_rf,0.564925,"{'model__max_depth': 3, 'model__max_features':..."
19,baseline_FEAR,balanced_rf,0.564693,"{'model__max_depth': 3, 'model__max_features':..."
7,baseline_socf,balanced_rf,0.546526,"{'model__max_depth': 3, 'model__max_features':..."
3,baseline_bs,balanced_rf,0.545900,"{'model__max_depth': 3, 'model__max_features':..."
17,baseline_WSO,balanced_rf,0.521815,"{'model__max_depth': 3, 'model__max_features':..."
5,baseline_dres,balanced_rf,0.502580,"{'model__max_depth': 3, 'model__max_features':..."
10,baseline_bmi,logistic,0.488383,"{'model__C': 0.01, 'model__penalty': 'l2', 'mo..."
6,baseline_socf,logistic,0.478051,"{'model__C': 1.0, 'model__penalty': 'l2', 'mod..."


Best logistic predictor: baseline_FEAR (score 0.602)
Best balanced RF predictor: baseline_dep (score 0.565)
Best-univariate threshold tuning (balanced accuracy):


,threshold,precision,recall,sensitivity,specificity,balanced_accuracy,accuracy,f1,tp,fp,fn,tn,constraint_met,model,splits
0,0.490,0.340909,0.833333,0.833333,0.442308,0.637821,0.542857,0.483871,15.0,29.0,3.0,23.0,False,logistic_baseline_FEAR,70
1,0.545,0.350000,0.388889,0.388889,0.750000,0.569444,0.657143,0.368421,7.0,13.0,11.0,39.0,False,balanced_rf_baseline_dep,70


Top thresholds (logistic best univariate)


,threshold,precision,recall,sensitivity,specificity,balanced_accuracy,accuracy,f1,tp,fp,fn,tn
98,0.490,0.340909,0.833333,0.833333,0.442308,0.637821,0.542857,0.483871,15,29,3,23
99,0.495,0.340909,0.833333,0.833333,0.442308,0.637821,0.542857,0.483871,15,29,3,23
104,0.520,0.461538,0.333333,0.333333,0.865385,0.599359,0.728571,0.387097,6,7,12,45
97,0.485,0.306122,0.833333,0.833333,0.346154,0.589744,0.471429,0.447761,15,34,3,18
100,0.500,0.315789,0.666667,0.666667,0.500000,0.583333,0.542857,0.428571,12,26,6,26
101,0.505,0.305556,0.611111,0.611111,0.519231,0.565171,0.542857,0.407407,11,25,7,27
94,0.470,0.278689,0.944444,0.944444,0.153846,0.549145,0.357143,0.430380,17,44,1,8
95,0.475,0.266667,0.888889,0.888889,0.153846,0.521368,0.342857,0.410256,16,44,2,8
93,0.465,0.265625,0.944444,0.944444,0.096154,0.520299,0.314286,0.414634,17,47,1,5
103,0.515,0.272727,0.333333,0.333333,0.692308,0.512821,0.600000,0.300000,6,16,12,36


Top thresholds (balanced RF best univariate)


,threshold,precision,recall,sensitivity,specificity,balanced_accuracy,accuracy,f1,tp,fp,fn,tn
109,0.545,0.350000,0.388889,0.388889,0.750000,0.569444,0.657143,0.368421,7,13,11,39
110,0.550,0.350000,0.388889,0.388889,0.750000,0.569444,0.657143,0.368421,7,13,11,39
128,0.640,0.600000,0.166667,0.166667,0.961538,0.564103,0.757143,0.260870,3,2,15,50
129,0.645,0.600000,0.166667,0.166667,0.961538,0.564103,0.757143,0.260870,3,2,15,50
125,0.625,0.500000,0.166667,0.166667,0.942308,0.554487,0.742857,0.250000,3,3,15,49
126,0.630,0.500000,0.166667,0.166667,0.942308,0.554487,0.742857,0.250000,3,3,15,49
127,0.635,0.500000,0.166667,0.166667,0.942308,0.554487,0.742857,0.250000,3,3,15,49
118,0.590,0.400000,0.222222,0.222222,0.884615,0.553419,0.714286,0.285714,4,6,14,46
108,0.540,0.318182,0.388889,0.388889,0.711538,0.550214,0.628571,0.350000,7,15,11,37
106,0.530,0.307692,0.444444,0.444444,0.653846,0.549145,0.600000,0.363636,8,18,10,34


In [53]:
# Accuracy, sensitivity, specificity for every univariate predictor
# WARNING: this can be slow (LOO threshold tuning per predictor).
# Exploratory default: skip unless explicitly enabled.
RUN_FULL_UNIVARIATE_METRICS = False

if not RUN_FULL_UNIVARIATE_METRICS:
    print("Skipping full per-predictor threshold tuning. Set RUN_FULL_UNIVARIATE_METRICS=True to run.")
else:
    log_rows, brf_rows = [], []

    for pred, (search, X_vals) in log_uni_searches.items():
        summary, _ = tune_balanced_accuracy(search.best_estimator_, X_vals, y, groups, pred)
        log_rows.append(
            {
                "predictor": pred,
                "threshold": summary["threshold"],
                "accuracy": summary["accuracy"],
                "sensitivity": summary["recall"],
                "specificity": summary["specificity"],
            }
        )

    for pred, (search, X_vals) in brf_uni_searches.items():
        summary, _ = tune_balanced_accuracy(search.best_estimator_, X_vals, y, groups, pred)
        brf_rows.append(
            {
                "predictor": pred,
                "threshold": summary["threshold"],
                "accuracy": summary["accuracy"],
                "sensitivity": summary["recall"],
                "specificity": summary["specificity"],
            }
        )

    log_metrics_df = pd.DataFrame(log_rows).sort_values("predictor").reset_index(drop=True)
    brf_metrics_df = pd.DataFrame(brf_rows).sort_values("predictor").reset_index(drop=True)

    print("Univariate logistic regression (threshold tuned for balanced accuracy):")
    display(log_metrics_df)

    print("Univariate balanced random forest (threshold tuned for balanced accuracy):")
    display(brf_metrics_df)

Skipping full per-predictor threshold tuning. Set RUN_FULL_UNIVARIATE_METRICS=True to run.
